# Devops News With Ollama

## Preparing the Environment
### Required Libraries
The project requires the installation of the following libraries:

```bash
pip install requests beautifulsoup4 python-dotenv openai ipykernel ipython
```

In [28]:
import requests
from bs4 import BeautifulSoup

from openai import OpenAI

from IPython.display import Markdown, display
from dotenv import load_dotenv

import os
load_dotenv()

True

### Constants
We define the constants that we will use in the project.

In [29]:
MODEL = "gemma-3n-e2b-it-text"
WEB_URL = "https://devopscube.com/blog/"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(base_url="http://192.168.0.12:1234/v1", api_key=OPENAI_API_KEY)

## Clase Scrapper



In [30]:
class Scrapper:
    """
    Utility for fetching and parsing HTML from a given URL.

    The Scrapper fetches an HTTP(S) resource, removes common irrelevant tags
    (script, style, img, input, nav, footer, header, aside), and exposes the
    parsed BeautifulSoup object plus convenient attributes:

    Attributes
    ----------
    url : str or None
        The last URL passed to get_content(), or None if nothing fetched.
    content : bs4.BeautifulSoup or None
        Parsed HTML of the fetched page.
    title : str or None
        Page title (derived from content) or 'No Title Found' when absent.
    body : str or None
        Page textual content with elements separated by newlines.
    links : list[str] or None
        List of href values from anchor tags in the page.

    Notes
    -----
    - get_content(url) performs the network request and populates the attributes.
    - Helper methods prefixed with an underscore (_get_title, _get_text, _get_links)
      are used internally to extract data from the parsed content.
    - This class expects requests and BeautifulSoup to be available in the environment.
    """

    def __init__(self):
        """
        Initialize an empty Scrapper instance.

        All attributes are initialized to None until get_content(url) is called.
        """
        self.url = None
        self.content = None
        self.title = None
        self.body = None
        self.links = None

    def get_content(self, url):
        """
        Fetch and parse HTML content from the given URL and populate attributes.

        Parameters
        ----------
        url : str
            The URL to fetch. Must start with 'http://' or 'https://'.

        Returns
        -------
        None
            The parsed BeautifulSoup object is stored in self.content and other
            derived attributes are populated; nothing is returned.

        Raises
        ------
        ValueError
            If the provided URL is invalid or if the fetched resource does not
            contain valid HTML content (no title and no text).

        Side effects
        ------------
        - Performs an HTTP GET using requests.get(url).
        - Removes the following tags from the parsed tree: script, style, img,
          input, nav, footer, header, aside.
        - Sets self.url, self.content, self.title, self.body, and self.links.
        """
        if url is None or not url.startswith(('http://', 'https://')):
            raise ValueError("A valid URL must be provided.")

        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        for irrelevant in soup(['script', 'style', 'img', 'input', 'nav', 'footer', 'header', 'aside']):
            irrelevant.decompose()

        if (not soup.title) and (not soup.get_text(strip=True)):
            raise ValueError("The URL does not contain valid HTML content.")

        self.url = url
        self.content = soup
        self.title = self._get_title()
        self.body = self._get_text()
        self.links = self._get_links()

    def _get_title(self):
        """
        Extract the page title from the parsed content.

        Returns
        -------
        str
            The title string if present, otherwise 'No Title Found'.

        Raises
        ------
        ValueError
            If no content has been fetched (self.content is None).
        """
        if not self.content:
            raise ValueError("Content is not fetched. Please fetch content with get_content method.")

        return self.content.title.string if self.content.title else 'No Title Found'

    def _get_text(self):
        """
        Extract the main textual content from the parsed HTML.

        Returns
        -------
        str
            The document text with elements separated by newline characters.

        Raises
        ------
        ValueError
            If no content has been fetched (self.content is None).
        """
        if not self.content:
            raise ValueError("Content is not fetched. Please fetch content with get_content method.")

        return self.content.get_text(separator='\n', strip=True)

    def _get_links(self):
        """
        Extract all hyperlinks (href attributes) from anchor tags.

        Returns
        -------
        list[str]
            A list of href values found in <a> tags.

        Raises
        ------
        ValueError
            If no content has been fetched (self.content is None).
        """
        if not self.content:
            raise ValueError("Content is not fetched. Please fetch content with get_content method.")

        return [a['href'] for a in self.content.find_all('a', href=True)]

In [31]:
devops_news =  Scrapper()
news_content = devops_news.get_content(WEB_URL)

print("-" * 80)
print(devops_news.title)
print("-" * 80)
print(devops_news.body[:500])  # Print first 500 characters of body
print("-" * 80)
print(devops_news.links[:5])   # Print first 5 links
print("-" * 80)

--------------------------------------------------------------------------------
DevOpsCube – Easy DevOps, SRE Guides & Reviews (Page 1)
--------------------------------------------------------------------------------
DevOpsCube – Easy DevOps, SRE Guides & Reviews (Page 1)
Blog
COMMON
How to Use Argo CD Image Updater with AWS ECR (Step-by-Step Guide)
In this step by step guide, we will look in to Argo CD Image Updater and how to use it
Aswin Vijayan
Sep 13, 2025
12 min read
AWS
How to Trigger an AWS Lambda Function using an EventBridge Rule
In this guide, we will build an event-driven architecture using AWS EventBridge and AWS Lambda.

You will learn how to
Arun Lal
Sep 12, 2025
7 min read
AWS EKS
How to Use 
--------------------------------------------------------------------------------
['https://devopscube.com/setup-argocd-image-updater/', '/tag/uncategorized-67d1aedf37a2cd001b59f5de/', 'https://devopscube.com/setup-argocd-image-updater/', '/author/aswin/', '/author/aswin/']
-------

# AI Summarizer Configuration
We are going to use the Ollama library to interact with a local LLM model.

In this section, we define the prompt template and the function to call the model.

### Prompt Template

In [37]:
system_prompt = """
    Eres un asistente que analiza el contenido de un sitio web
    y proporciona un breve resumen, en español, ignorando el texto que podría estar relacionado con la navegación.
    Responder en Markdown.
"""

user_prompt = f"""
    Te envío a continuación el texto de un sitio web y los links obtenidos del mismo, titulado {devops_news.title}.
    - Proporciona un breve resumen de cada publicación del blog en una tabla en formato Markdown.
    - Destaca la noticia, una breve descripción y el enlace para poder acceder a la noticia.
    
    El contenido de este sitio web es el siguiente:
    {devops_news.body}
    
    Los links extraídos son:
    {devops_news.links}
"""

print(user_prompt)


    Te envío a continuación el texto de un sitio web y los links obtenidos del mismo, titulado DevOpsCube – Easy DevOps, SRE Guides & Reviews (Page 1).
    - Proporciona un breve resumen de cada publicación del blog en una tabla en formato Markdown.
    - Destaca la noticia, una breve descripción y el enlace para poder acceder a la noticia.

    El contenido de este sitio web es el siguiente:
    DevOpsCube – Easy DevOps, SRE Guides & Reviews (Page 1)
Blog
COMMON
How to Use Argo CD Image Updater with AWS ECR (Step-by-Step Guide)
In this step by step guide, we will look in to Argo CD Image Updater and how to use it
Aswin Vijayan
Sep 13, 2025
12 min read
AWS
How to Trigger an AWS Lambda Function using an EventBridge Rule
In this guide, we will build an event-driven architecture using AWS EventBridge and AWS Lambda.

You will learn how to
Arun Lal
Sep 12, 2025
7 min read
AWS EKS
How to Use EFS on EKS with the EFS CSI Driver
In this guide, you will learn to provision Amazon EFS as persist

### System and User Prompts

In [38]:
prompt_message = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

### Results
Use the OpenAI SDK call to get the summary of the website content.

In [39]:
summary = client.chat.completions.create(model=MODEL, messages=prompt_message, temperature=0.4)

In [40]:
print(f"Prompt tokens used: {summary.usage.prompt_tokens}")
print(f"Completion tokens used: {summary.usage.completion_tokens}")
print(f"Total tokens used: {summary.usage.total_tokens}")

display(Markdown(summary.choices[0].message.content))

Prompt tokens used: 3202
Completion tokens used: 4308
Total tokens used: 7510


## Resumen de Publicaciones del Blog DevOpsCube

| Noticia | Descripción | Enlace |
|---|---|---|
| **How to Use Argo CD Image Updater with AWS ECR** | Guía paso a paso sobre cómo utilizar el Argo CD Image Updater para gestionar las imágenes en AWS ECR. | [https://devopscube.com/setup-argocd-image-updater/](https://devopscube.com/setup-argocd-image-updater/) |
| **How to Trigger an AWS Lambda Function using an EventBridge Rule** | Explica cómo crear una arquitectura basada en AWS EventBridge y Lambda para disparar funciones Lambda mediante reglas de eventos. | [https://devopscube.com/trigger-aws-lambda-function-using-event-bridge-rule/](https://devopscube.com/trigger-aws-lambda-function-using-event-bridge-rule/) |
| **How to Use EFS on EKS with the EFS CSI Driver** |  Detalla la configuración de Amazon EFS como almacenamiento persistente para un clúster Amazon EKS. | [https://devopscube.com/setup-efs-csi-driver-eks/](https://devopscube.com/setup-efs-csi-driver-eks/) |
| **How to Mount OCI Image as Volume in Kubernetes Pods** |  Enseña cómo montar imágenes de OCI como volúmenes en los pods de Kubernetes. | [https://devopscube.com/oci-image-volume-kubernetes-pods/](https://devopscube.com/oci-image-volume-kubernetes-pods/) |
| **Amazon Q CLI: A Practical Guide to Getting Started** |  Presenta la Amazon Q Developer CLI y sus beneficios para evitar escribir comandos complejos de AWS CLI. | [https://devopscube.com/set-up-amazon-q-cli/](https://devopscube.com/set-up-amazon-q-cli/) |
| **Cloudways Promo Codes & Offers For 2025** |  Lista los códigos promocionales y ofertas más recientes de Cloudways para obtener descuentos en sus servicios de hosting gestionado. | [https://devopscube.com/cloudways-promo-codes/](https://devopscube.com/cloudways-promo-codes/) |
| **Observability as Code with Grafana Git Sync** |  Explica cómo integrar la versionado de código con Grafana dashboards para mantener la observability en producción. | [https://devopscube.com/observability-as-code-with-grafana-git-sync/](https://devopscube.com/observability-as-code-with-grafana-git-sync/) |
| **Cloudways Review 2025: Details, Pricing, & Features** |  Una revisión detallada de Cloudways, incluyendo características, precios y ventajas y desventajas. | [https://devopscube.com/cloudways-review/](https://devopscube.com/cloudways-review/) |
| **KodeKloud Coupons and Promotions for 2025** |  Proporciona códigos de descuento y promociones actuales para KodeKloud, una plataforma de aprendizaje de DevOps. | [https://devopscube.com/kodekloud-coupons/](https://devopscube.com/kodekloud-coupons/) |
| **[55% OFF] Kubernetes Certification Coupon: $1685 Discount On Kubernetes Bundles** |  Un código de descuento para la certificación CKA, ofreciendo un ahorro significativo en los paquetes de Kubernetes. | [https://devopscube.com/kubernetes-certification-coupon/](https://devopscube.com/kubernetes-certification-coupon/) |
| **Linux Foundation Coupon: 80% Off For DevOps Enthusiasts** |  Un código de descuento para las certificaciones de la Linux Foundation, con un descuento del 80%. | [https://devopscube.com/linux-foundation-coupon/](https://devopscube.com/linux-foundation-coupon/) |
| **[75% Off ] Udacity Coupons 2025: Save Big On Nanodegree Programs** |  Códigos de descuento y promociones para los nanodegrees de Udacity, con un descuento del 75%. | [https://devopscube.com/udacity-coupon-code/](https://devopscube.com/udacity-coupon-code/) |
| **How to Set Up ExternalDNS on EKS?** |  Guía paso a paso para configurar ExternalDNS en un clúster Amazon EKS y automatizar la creación de registros DNS. | [https://devopscube.com/setup-externaldns-on-eks/](https://devopscube.com/setup-externaldns-on-eks/) |
| **Become A DevOps Engineer in 2025: [A Practical Roadmap]** |  Una guía práctica para convertirse en ingeniero DevOps en 2025, basada en 10 años de experiencia. | [https://devopscube.com/become-devops-engineer/](https://devopscube.com/become-devops-engineer/) |
| **Pass the CKA Certification Exam: The Ultimate Study Guide** |  Una guía completa con consejos, guías temáticas y recursos para prepararse para el examen de certificación CKA. | [https://devopscube.com/cka-exam-study-guide/](https://devopscube.com/cka-exam-study-guide/) |
| **Kubernetes Pod Guide: Core Concepts with Practical Examples** |  Una guía detallada de los conceptos centrales de los pods en Kubernetes, con ejemplos prácticos y casos de uso. | [https://devopscube.com/kubernetes-pod/](https://devopscube.com/kubernetes-pod/) |
| **Setup MySQL Operator on Kubernetes (Step by Step Guide)** |  Una guía paso a paso para configurar y operar el operador de MySQL en un clúster Kubernetes. | [https://devopscube.com/install-mysql-operator-on-kubernetes/](https://devopscube.com/install-mysql-operator-on-kubernetes/) |
| **Kubernetes Daemonset: A Comprehensive Guide** |  Una guía completa para entender los DaemonSets en Kubernetes, ideal para principiantes. | [https://devopscube.com/kubernetes-daemonset/](https://devopscube.com/kubernetes-daemonset/) |
| **KCNA Certification: Detailed Study Guide** | Una guía de estudio detallada para la certificación KCNA (Kubernetes and Cloud Native Associate). | [https://devopscube.com/kcna-study-guide/](https://devopscube.com/kcna-study-guide/) |
| **KCSA Certification: The Complete Study Guide With Resources** |  Un estudio completo con recursos para la certificación KCSA (Kubernetes and Cloud Native Security Associate). | [https://devopscube.com/kcsa-exam-study-guide/](https://devopscube.com/kcsa-exam-study-guide/) |
| **Automate Adding Clusters to ArgoCD using GitHub Actions** |  Cómo automatizar la adición de clusters a ArgoCD usando flujos de trabajo de GitHub Actions. | [https://devopscube.com/automate-adding-clusters-to-argocd/](https://devopscube.com/automate-adding-clusters-to-argocd/) |
| **How to Dockerize a Golang Application (Step-by-Step Guide)** |  Una guía paso a paso y amigable para principiantes sobre cómo Dockerizar una aplicación Go. | [https://devopscube.com/dockerize-a-golang-application/](https://devopscube.com/dockerize-a-golang-application/) |
| **Kubernetes Pod Graceful Shutdown with SIGTERM & preStop Hooks** |  Cómo manejar el apagado gracioso de pods de Kubernetes usando SIGTERM y hooks preStop, con ejemplos prácticos. | [https://devopscube.com/kubernetes-pod-graceful-shutdown/](https://devopscube.com/kubernetes-pod-graceful-shutdown/) |
| **Kubernetes Architecture: The Definitive Guide (2025)** |  Una guía completa e ilustrada sobre la arquitectura de Kubernetes, desde adentro hacia afuera. | [https://devopscube.com/kubernetes-architecture-explained/](https://devopscube.com/kubernetes-architecture-explained/) |
| **Setup ExternalDNS on EKS?** | Cómo instalar y configurar ExternalDNS en un clúster Amazon EKS para automatizar la creación de registros DNS. | [https://devopscube.com/setup-externaldns-on-eks/](https://devopscube.com/setup-externaldns-on-eks/) |
| **How to Set Up ExternalDNS on EKS?** | Cómo instalar y configurar ExternalDNS en un clúster Amazon EKS para automatizar la creación de registros DNS. | [https://devopscube.com/setup-externaldns-on-eks/](https://devopscube.com/setup-externaldns-on-eks/) |
| **Amazon Q CLI: A Practical Guide to Getting Started** |  Una guía práctica para comenzar a usar Amazon Q Developer CLI y evitar escribir largas y repetitivas comandos AWS CLI. | [https://devopscube.com/set-up-amazon-q-cli/](https://devopscube.com/set-up-amazon-q-cli/) |
| **Cloudways Review 2025: Details, Pricing, & Features** | Una revisión detallada de Cloudways con precios, funcionalidades y pros y contras para ayudarte a elegir la mejor opción. | [https://devopscube.com/cloudways-review/](https://devopscube.com/cloudways-review/) |
| **Observability as Code with Grafana Git Sync** | Explora cómo implementar observability como código usando Grafana y sincronización con Git para gestionar dashboards de forma eficiente. | [https://devopscube.com/observability-as-code-with-grafana-git-sync/](https://devopscube.com/observability-as-code-with-grafana-git-sync/) |
| **How to Mount OCI Image as Volume in Kubernetes Pods** | Aprende cómo montar imágenes de OCI como volúmenes en pods de Kubernetes para una gestión de datos flexible y eficiente. | [https://devopscube.com/oci-image-volume-kubernetes-pods/](https://devopscube.com/oci-image-volume-kubernetes-pods/) |
| **Become A DevOps Engineer in 2025: [A Practical Roadmap]** | Un roadmap práctico para convertirse en ingeniero DevOps en 2025, basado en 10 años de experiencia y las mejores prácticas.  | [https://devopscube.com/become-devops-engineer/](https://devopscube.com/become-devops-engineer/) |
| **Pass the CKA Certification Exam: The Ultimate Study Guide** | Una guía de estudio completa para prepararte para el examen Certified Kubernetes Administrator (CKA) y dominar los conceptos clave. | [https://devopscube.com/cka-exam-study-guide/](https://devopscube.com/cka-exam-study-guide/) |
| **Kubernetes Pod Guide: Core Concepts with Practical Examples** | Aprende los conceptos fundamentales de los pods de Kubernetes con ejemplos prácticos y casos de uso para una mejor comprensión. | [https://devopscube.com/kubernetes-pod/](https://devopscube.com/kubernetes-pod/) |
| **Setup MySQL Operator on Kubernetes (Step by Step Guide)** | Aprende paso a paso cómo configurar y operar el operador de MySQL en un clúster Kubernetes para una gestión eficiente de bases de datos. | [https://devopscube.com/install-mysql-operator-on-kubernetes/](https://devopscube.com/install-mysql-operator-on-kubernetes/) |
| **Kubernetes Daemonset: A Comprehensive Guide** | Una guía completa para entender los DaemonSets en Kubernetes, ideal para principiantes que desean aprender más sobre este recurso. | [https://devopscube.com/kubernetes-daemonset/](https://devopscube.com/kubernetes-daemonset/) |
| **KCNA Certification: Detailed Study Guide** | Una guía de estudio detallada para prepararte para el examen KCNA (Kubernetes and Cloud Native Associate) y dominar los conceptos clave. | [https://devopscube.com/kcna-study-guide/](https://devopscube.com/kcna-study-guide/) |
| **KCSA Certification: The Complete Study Guide With Resources** | Una guía completa con recursos para prepararte para el examen KCSA (Kubernetes and Cloud Native Security Associate) y dominar los conceptos clave. | [https://devopscube.com/kcsa-exam-study-guide/](https://devopscube.com/kcsa-exam-study-guide/) |
| **Automate Adding Clusters to ArgoCD using GitHub Actions** | Aprende cómo automatizar la adición de clusters a ArgoCD usando workflows de GitHub Actions para simplificar la gestión y reducir errores. | [https://devopscube.com/automate-adding-clusters-to-argocd/](https://devopscube.com/automate-adding-clusters-to-argocd/) |
| **How to Dockerize a Golang Application (Step-by-Step Guide)** | Aprende cómo empaquetar una aplicación Go con Docker paso a paso, desde la configuración inicial hasta la creación de un Dockerfile eficiente. | [https://devopscube.com/dockerize-a-golang-application/](https://devopscube.com/dockerize-a-golang-application/) |
| **Kubernetes Pod Graceful Shutdown with SIGTERM & preStop Hooks** | Aprende cómo gestionar el apagado gracioso de pods en Kubernetes usando SIGTERM y hooks preStop para minimizar interrupciones. | [https://devopscube.com/kubernetes-pod-graceful-shutdown/](https://devopscube.com/kubernetes-pod-graceful-shutdown/) |
| **Kubernetes Architecture: The Definitive Guide (2025)** | Explora la arquitectura de Kubernetes en detalle, desde los componentes básicos hasta las funcionalidades avanzadas, con ejemplos prácticos y diagramas ilustrativos. | [https://devopscube.com/kubernetes-architecture-explained/](https://devopscube.com/kubernetes-architecture-explained/) |
| **How to Set Up ExternalDNS on EKS?** | Aprende cómo instalar y configurar ExternalDNS en Amazon EKS para automatizar la gestión de registros DNS, facilitando el despliegue de aplicaciones en la nube. | [https://devopscube.com/setup-externaldns-on-eks/](https://devopscube.com/setup-externaldns-on-eks/) |
| **Become A DevOps Engineer in 2025: [A Practical Roadmap]** | Sigue una guía práctica con 10 años de experiencia para convertirte en un ingeniero DevOps, cubriendo las habilidades y herramientas esenciales que necesitas dominar. | [https://devopscube.com/become-devops-engineer/](https://devopscube.com/become-devops-engineer/) |
| **Pass the CKA Certification Exam: The Ultimate Study Guide** | Prepara el examen de Certified Kubernetes Administrator (CKA) con esta guía completa, que incluye consejos expertos, guías temáticas y recursos prácticos para ayudarte a aprobar. | [https://devopscube.com/cka-exam-study-guide/](https://devopscube.com/cka-exam-study-guide/) |
| **Kubernetes Pod Guide: Core Concepts with Practical Examples** | Explica los conceptos básicos de los pods en Kubernetes con ejemplos prácticos y casos de uso, para que puedas comprender cómo funcionan y cómo utilizarlos eficazmente. | [https://devopscube.com/kubernetes-pod/](https://devopscube.com/kubernetes-pod/) |
| **Setup MySQL Operator on Kubernetes (Step by Step Guide)** | Aprende a instalar y operar el MySQL Operator en un clúster de Kubernetes, para que puedas gestionar tus bases de datos MySQL de forma eficiente. | [https://devopscube.com/install-mysql-operator-on-kubernetes/](https://devopscube.com/install-mysql-operator-on-kubernetes/) |
| **Kubernetes Daemonset: A Comprehensive Guide** | Descubre todo lo que necesitas saber sobre los DaemonSets en Kubernetes, desde los conceptos básicos hasta las configuraciones avanzadas, con ejemplos prácticos y casos de uso. | [https://devopscube.com/kubernetes-daemonset/](https://devopscube.com/kubernetes-daemonset/) |
| **KCNA Certification: Detailed Study Guide** |  Esta guía detallada te ayudará a prepararte para el examen KCNA (Kubernetes and Cloud Native Associate). Cubre los temas importantes y ofrece consejos prácticos para aprobar. | [https://devopscube.com/kcna-study-guide/](https://devopscube.com/kcna-study-guide/) |
| **KCSA Certification: The Complete Study Guide With Resources** |  Prepárate para el examen KCSA (Kubernetes and Cloud Native Security Associate) con esta guía completa, que incluye recursos adicionales y consejos para optimizar tu estudio. | [https://devopscube.com/kcsa-exam-study-guide/](https://devopscube.com/kcsa-exam-study-guide/) |
| **Automate Adding Clusters to ArgoCD using GitHub Actions** | Aprende a automatizar la adición de clusters a ArgoCD utilizando workflows de GitHub Actions, para simplificar la gestión y evitar errores manuales. | [https://devopscube.com/automate-adding-clusters-to-argocd/](https://devopscube.com/automate-adding-clusters-to-argocd/) |
| **How to Dockerize a Golang Application (Step-by-Step Guide)** |  Sigue este tutorial paso a paso para empaquetar tu aplicación Go en un contenedor Docker, de forma sencilla y práctica. | [https://devopscube.com/dockerize-a-golang-application/](https://devopscube.com/dockerize-a-golang-application/) |
| **Kubernetes Pod Graceful Shutdown with SIGTERM & preStop Hooks** | Aprende a manejar el apagado gracioso de pods en Kubernetes utilizando SIGTERM y hooks preStop, para evitar interrupciones inesperadas. | [https://devopscube.com/kubernetes-pod-graceful-shutdown/](https://devopscube.com/kubernetes-pod-graceful-shutdown/) |
| **Kubernetes Architecture: The Definitive Guide (2025)** | Explora la arquitectura de Kubernetes desde adentro hacia afuera, con una guía ilustrada que te permite comprender cada componente y su función. | [https://devopscube.com/kubernetes-architecture-explained/](https://devopscube.com/kubernetes-architecture-explained/) |


# Summarizing Blogs Content
We  are going to summarize the content of the blogs found in the DevOps Cube website using a local LLM model hosted with LM Studio.